# Universal Worker

> FastAPI server that runs inside isolated plugin environments

In [ ]:
#| default_exp core.worker

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import argparse
import dataclasses
import importlib
import json
import os
import signal
import sys
import threading
import time
from typing import Any, Dict, Generator

import psutil
import uvicorn
from fastapi import FastAPI, HTTPException, Request
from fastapi.responses import StreamingResponse

import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    force=True
)

The Universal Worker is a FastAPI server that:

1. **Dynamically loads** a plugin class specified via CLI arguments
2. **Exposes HTTP endpoints** for plugin lifecycle and execution
3. **Monitors parent process** ("Suicide Pact") to prevent zombie workers
4. **Reports telemetry** for resource scheduling decisions

```
Host Process                    Worker Process (Isolated Env)
┌──────────────┐               ┌─────────────────────────────┐
│ RemoteProxy  │──HTTP/JSON───▶│ Universal Worker (FastAPI)  │
│              │               │   ├─ /health                │
│  --ppid ─────│───────────────│──▶│ ├─ /initialize          │
│              │               │   ├─ /execute               │
└──────────────┘               │   ├─ /execute_stream        │
                               │   └─ PID Watchdog (Thread)  │
                               └─────────────────────────────┘
```

## EnhancedJSONEncoder

Custom JSON encoder that handles dataclasses and other common Python types that need serialization for HTTP responses.

In [ ]:
#| export
class EnhancedJSONEncoder(json.JSONEncoder):
    """JSON encoder that handles dataclasses and other common types."""
    
    def default(
        self,
        o: Any # Object to encode
    ) -> Any: # JSON-serializable representation
        """Convert non-serializable objects to serializable form."""
        if dataclasses.is_dataclass(o) and not isinstance(o, type):
            return dataclasses.asdict(o)
        return super().default(o)

In [ ]:
# Test EnhancedJSONEncoder
from dataclasses import dataclass

@dataclass
class SampleConfig:
    name: str
    value: int

cfg = SampleConfig(name="test", value=42)
result = json.dumps(cfg, cls=EnhancedJSONEncoder)
print(f"Encoded: {result}")
assert json.loads(result) == {"name": "test", "value": 42}

Encoded: {"name": "test", "value": 42}


## PID Watchdog

The "Suicide Pact" pattern: if the Host process dies, the Worker must terminate itself to prevent zombie processes consuming resources.

In [ ]:
#| export
def parent_monitor(
    ppid: int # Parent process ID to monitor
) -> None:
    """Monitor parent process and terminate self if parent dies."""
    try:
        while True:
            parent = psutil.Process(ppid)
            if parent.status() == psutil.STATUS_ZOMBIE:
                raise psutil.NoSuchProcess(ppid)
            time.sleep(1)
    except (psutil.NoSuchProcess, ProcessLookupError):
        print(f"[Worker {os.getpid()}] Parent {ppid} gone. Shutting down.")
        os.kill(os.getpid(), signal.SIGTERM)

The watchdog runs in a daemon thread, checking every second if the parent process is still alive. When the parent dies (or becomes a zombie), the worker sends `SIGTERM` to itself for graceful shutdown.

## Application Factory

Creates the FastAPI application with all endpoints for plugin communication.

In [ ]:
#| export
def create_app(
    module_name: str, # Python module path (e.g., "my_plugin.plugin")
    class_name: str   # Plugin class name (e.g., "WhisperPlugin")
) -> FastAPI: # Configured FastAPI application
    """Create FastAPI app that hosts the specified plugin."""
    app = FastAPI(title="Plugin Worker")
    plugin_instance = None

    # Dynamic Loading
    try:
        module = importlib.import_module(module_name)
        plugin_cls = getattr(module, class_name)
        plugin_instance = plugin_cls()
    except Exception as e:
        print(f"FATAL: Failed to load {module_name}:{class_name} - {e}")
        sys.exit(1)

    @app.get("/health")
    def health_check() -> Dict[str, Any]:
        """Health check endpoint."""
        return {
            "status": "running",
            "pid": os.getpid(),
            "name": getattr(plugin_instance, "name", "unknown"),
            "version": getattr(plugin_instance, "version", "unknown")
        }

    @app.get("/stats")
    def stats() -> Dict[str, Any]:
        """Return process tree resource usage."""
        proc = psutil.Process()
        
        # 1. Get Memory of Main Process
        total_rss = proc.memory_info().rss
        
        # 2. Get Memory of Children (Safe Loop)
        # recursive=True ensures we catch grandchildren
        for child in proc.children(recursive=True):
            try:
                # We must check if child still exists before asking for memory
                total_rss += child.memory_info().rss
            except (psutil.NoSuchProcess, psutil.AccessDenied):
                # Process died or we don't have permission (rare for child), skip it
                pass

        return {
            "pid": os.getpid(),
            "cpu_percent": proc.cpu_percent(), # Note: aggregating CPU % is complex, main pid is usually enough proxy
            "memory_rss_mb": total_rss / 1024 / 1024
        }

    @app.post("/initialize")
    async def initialize(request: Request) -> Dict[str, str]:
        """Initialize or reconfigure the plugin."""
        try:
            config = await request.json()
            plugin_instance.initialize(config)
            return {"status": "ok"}
        except Exception as e:
            raise HTTPException(status_code=400, detail=str(e))

    @app.get("/config_schema")
    def get_config_schema() -> Dict[str, Any]:
        """Return JSON Schema for plugin configuration."""
        if hasattr(plugin_instance, 'get_config_schema'):
            return plugin_instance.get_config_schema()
        return {}

    @app.get("/config")
    def get_config() -> Dict[str, Any]:
        """Return current plugin configuration."""
        if hasattr(plugin_instance, 'get_current_config'):
            cfg = plugin_instance.get_current_config()
            # Ensure dataclasses are serialized
            return json.loads(json.dumps(cfg, cls=EnhancedJSONEncoder))
        return {}

    @app.post("/execute")
    async def execute(request: Request) -> Any:
        """Execute plugin's main functionality."""
        data = await request.json()
        args = data.get("args", [])
        kwargs = data.get("kwargs", {})
        
        try:
            result = plugin_instance.execute(*args, **kwargs)
            # Serialize result (handles dataclasses)
            json_str = json.dumps(result, cls=EnhancedJSONEncoder)
            return json.loads(json_str)
        except Exception as e:
            import traceback
            traceback.print_exc()
            raise HTTPException(status_code=500, detail=str(e))

    @app.post("/execute_stream")
    async def execute_stream(request: Request) -> StreamingResponse:
        """Execute plugin with streaming response (NDJSON)."""
        data = await request.json()
        args = data.get("args", [])
        kwargs = data.get("kwargs", {})

        def iter_response() -> Generator[str, None, None]:
            try:
                if hasattr(plugin_instance, 'execute_stream'):
                    iterator = plugin_instance.execute_stream(*args, **kwargs)
                else:
                    # Fallback: wrap single result
                    iterator = [plugin_instance.execute(*args, **kwargs)]
                
                for chunk in iterator:
                    # Line-delimited JSON (NDJSON)
                    yield json.dumps(chunk, cls=EnhancedJSONEncoder) + "\n"
            except Exception as e:
                yield json.dumps({"error": str(e)}) + "\n"

        return StreamingResponse(iter_response(), media_type="application/x-ndjson")

    @app.post("/cleanup")
    def cleanup() -> Dict[str, str]:
        """Clean up plugin resources."""
        if hasattr(plugin_instance, 'cleanup'):
            plugin_instance.cleanup()
        return {"status": "cleaned"}

    return app

### Endpoint Summary

| Endpoint | Method | Purpose |
|----------|--------|----------------------------------------------|
| `/health` | GET | Health check with PID and plugin identity |
| `/stats` | GET | Process telemetry (CPU, memory) for scheduler |
| `/initialize` | POST | Configure/reconfigure plugin |
| `/config_schema` | GET | JSON Schema for UI generation |
| `/config` | GET | Current configuration values |
| `/execute` | POST | Execute plugin, return JSON |
| `/execute_stream` | POST | Execute with streaming NDJSON response |
| `/cleanup` | POST | Release plugin resources |

## CLI Entry Point

The worker is launched as a subprocess with the plugin module/class and port specified via command line arguments.

In [ ]:
#| export
def run_worker() -> None:
    """CLI entry point for running the worker."""
    parser = argparse.ArgumentParser(description="Universal Plugin Worker")
    parser.add_argument("--module", required=True, help="Plugin module path")
    parser.add_argument("--class", dest="class_name", required=True, help="Plugin class name")
    parser.add_argument("--port", type=int, required=True, help="Port to listen on")
    parser.add_argument("--ppid", type=int, required=False, help="Parent PID to monitor")
    args = parser.parse_args()

    # Start watchdog if parent PID provided
    if args.ppid:
        watchdog = threading.Thread(
            target=parent_monitor,
            args=(args.ppid,),
            daemon=True
        )
        watchdog.start()

    app = create_app(args.module, args.class_name)
    uvicorn.run(app, host="127.0.0.1", port=args.port, log_level="warning")

### Usage

The worker is typically launched by the `RemotePluginProxy`:

```bash
# Example: Launch a Whisper plugin worker
python -m cjm_plugin_system.core.worker \
    --module cjm_transcription_plugin_whisper.plugin \
    --class WhisperLocalPlugin \
    --port 12345 \
    --ppid 1234
```

The `--ppid` argument enables the suicide pact: if process 1234 dies, this worker terminates.

In [ ]:
#| export
#| eval: false
import sys

if __name__ == "__main__" and "ipykernel" not in sys.modules:
    run_worker()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()